In [3]:
# reference code: https://www.guru99.com/seq2seq-model.html
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

import os
import re
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [63]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

class vocab_processing:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {0:'SOS', 1:'EOS',}
        self.word_count = {}
        self.n_words = 2
        
    def make_voc_dict(self, sentence):
        for word in sentence.split(' '):
            if word not in self.word2idx:
                self.word2idx[word] = self.n_words
                self.idx2word[self.n_words] = word
                self.word_count[word] = 1
                self.n_words += 1
            else:
                self.word_count[word] += 1

In [41]:
lang1 = 'eng'
lang2 = 'kor'
path = 'data/%s-%s.txt' % (lang1, lang2)
df = pd.read_csv(path, delimiter='\t', header=None, index_col=2, names=[lang1, lang2])

In [66]:
def process_data(lang1, lang2):
    """
    >> df
                                                                   eng  kor
        CC-BY 2.0 (France) Attribution: tatoeba.org #20...    Who?  누구?
        CC-BY 2.0 (France) Attribution: tatoeba.org #37...  Hello!  안녕!
    """
    
    path = 'data/%s-%s.txt' % (lang1, lang2)
    df = pd.read_csv(path, delimiter='\t', header=None, index_col=2, names=[lang1, lang2])
    
    def _process_text(lang):
        """
        https://towardsdatascience.com/difference-between-nfd-nfc-nfkd-and-nfkc-explained-with-python-code-e2631f96ae6c
        - remove special char
        - normalize space 
        """
        sentence = df[lang].str.lower()
        sentence = sentence.str.replace('[^A-Za-z\s]+', '')
        sentence = sentence.str.normalize('NFD')
        sentence = sentence.str.encode('ascii', errors='ignore').str.decode('utf-8')
        return sentence.values
        
    sentence1, sentence2 = _process_text(lang1), _process_text(lang2)
    source = vocab_processing()
    target = vocab_processing()
    pairs = []
    
    for i in range(len(df)):
        if len(sentence1[i].split(' ')) < MAX_LENGTH \
            and len(sentence2[i].split(' ')) < MAX_LENGTH:
            
            full = [sentence1[i], sentence2[i]]
            source.make_voc_dict(sentence1[i])
            target.make_voc_dict(sentence2[i])
            pairs.append(full)
            
    return source, target, pairs    
source, target, pairs = process_data(lang1, lang2)

In [78]:
def sen2tensor(lang, sentence):
    print(sentence)
    indexes = [lang.word2index[word] for word in sentence.split(' ')]
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def pair2tensor(input_lang, output_lang, pair):
    input_tensor = sen2tensor(input_lang, pair[0])
    target_tensor = sen2tensor(output_lang, pair[1])
    return (input_tensor, target_tensor)

pair2tensor(lang1, lang2, pairs)

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers):
        super(Encoder, self).__init__()

        # set the encoder input dimesion , embbed dimesion, hidden dimesion, and number of layers
        self.input_dim = input_dim
        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # initialize the embedding layer with input and embbed dimention
        self.embedding = nn.Embedding(input_dim, self.embbed_dim)
        # intialize the GRU to take the input dimetion of embbed, and output dimention of hidden and
        # set the number of gru layers
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim,
                          num_layers=self.num_layers)

    def forward(self, src):

        embedded = self.embedding(src).view(1, 1, -1)
        outputs, hidden = self.gru(embedded)
        return outputs, hidden

In [9]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers):
        super(Decoder, self).__init__()

# set the encoder output dimension, embed dimension, hidden dimension, and number of layers
        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

# initialize every layer with the appropriate dimension. For the decoder layer, it will consist of an embedding, GRU, a Linear layer and a Log softmax activation function.
        self.embedding = nn.Embedding(output_dim, self.embbed_dim)
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim,
                          num_layers=self.num_layers)
        self.out = nn.Linear(self.hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):

        # reshape the input to (1, batch_size)
        input = input.view(1, -1)
        embedded = F.relu(self.embedding(input))
        output, hidden = self.gru(embedded, hidden)
        prediction = self.softmax(self.out(output[0]))

        return prediction, hidden

In [10]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, MAX_LENGTH=MAX_LENGTH):
        super().__init__()

# initialize the encoder and decoder
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source, target, teacher_forcing_ratio=0.5):

        # get the input length (number of words in sentence)
        input_length = source.size(0)
        batch_size = target.shape[1]
        target_length = target.shape[0]
        vocab_size = self.decoder.output_dim

# initialize a variable to hold the predicted outputs
        outputs = torch.zeros(target_length, batch_size,
                              vocab_size).to(self.device)

# encode every word in a sentence
        for i in range(input_length):
            encoder_output, encoder_hidden = self.encoder(source[i])

# use the encoder’s hidden layer as the decoder hidden
        decoder_hidden = encoder_hidden.to(device)

# add a token before the first predicted word
        decoder_input = torch.tensor([SOS_token], device=device)  # SOS

# topk is used to get the top K value over a list
# predict the output word from the current target word. If we enable the teaching force,  then the #next decoder input is the next word, else, use the decoder output highest value.

        for t in range(target_length):
            decoder_output, decoder_hidden = self.decoder(
                decoder_input, decoder_hidden)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            topv, topi = decoder_output.topk(1)
            input = (target[t] if teacher_force else topi)
            if(teacher_force == False and input.item() == EOS_token):
                break

        return outputs

In [21]:
teacher_forcing_ratio = 0.5


def clacModel(model, input_tensor, target_tensor, model_optimizer, criterion):
    model_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    loss = 0
    epoch_loss = 0
    # print(input_tensor.shape)

    output = model(input_tensor, target_tensor)

    num_iter = output.size(0)
    print(num_iter)

# calculate the loss from a predicted sentence with the expected result
    for ot in range(num_iter):
        loss += criterion(output[ot], target_tensor[ot])

    loss.backward()
    model_optimizer.step()
    epoch_loss = loss.item() / num_iter

    return epoch_loss


def trainModel(model, source, target, pairs, num_iteration=20000):
    model.train()

    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.NLLLoss()
    total_loss_iterations = 0

    training_pairs = [tensorsFromPair(source, target, random.choice(pairs))
                      for i in range(num_iteration)]

    for iter in range(1, num_iteration+1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = clacModel(model, input_tensor, target_tensor,
                         optimizer, criterion)

        total_loss_iterations += loss

        if iter % 5000 == 0:
            avarage_loss = total_loss_iterations / 5000
            total_loss_iterations = 0
            print('%d %.4f' % (iter, avarage_loss))

    torch.save(model.state_dict(), 'mytraining.pt')
    return model

In [14]:
def evaluate(model, input_lang, output_lang, sentences, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentences[0])
        output_tensor = tensorFromSentence(output_lang, sentences[1])

        decoded_words = []

        output = model(input_tensor, output_tensor)
        # print(output_tensor)

        for ot in range(output.size(0)):
            topv, topi = output[ot].topk(1)
            # print(topi)

            if topi[0].item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi[0].item()])
    return decoded_words


def evaluateRandomly(model, source, target, pairs, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('source {}'.format(pair[0]))
        print('target {}'.format(pair[1]))
        output_words = evaluate(model, source, target, pair)
        output_sentence = ' '.join(output_words)
        print('predicted {}'.format(output_sentence))

In [22]:
lang1 = 'eng'
lang2 = 'kor'
source, target, pairs = process_data(lang1, lang2)

randomize = random.choice(pairs)
print('random sentence {}'.format(randomize))

#print number of words
input_size = source.n_words
output_size = target.n_words
print('Input : {} Output : {}'.format(input_size, output_size))

embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 10

#create encoder-decoder model
encoder = Encoder(input_size, hidden_size, embed_size, num_layers)
decoder = Decoder(output_size, hidden_size, embed_size, num_layers)

model = Seq2Seq(encoder, decoder, device).to(device)

#print model 
print(encoder)
print(decoder)

model = trainModel(model, source, target, pairs, num_iteration)
evaluateRandomly(model, source, target, pairs)

Read 904 sentence pairs
random sentence ['  ', 'ccby  france attribution tatoebaorg  ck   queserasera']
Input : 9 Output : 140
Encoder(
  (embedding): Embedding(9, 256)
  (gru): GRU(256, 512)
)
Decoder(
  (embedding): Embedding(140, 256)
  (gru): GRU(256, 512)
  (out): Linear(in_features=512, out_features=140, bias=True)
  (softmax): LogSoftmax()
)
11
11
11
11
11
11
11
11
11
11
source        
target ccby  france attribution tatoebaorg  trieuho   happymidnight
predicted           
source    
target ccby  france attribution tatoebaorg  jubjubbird   janason
predicted           
source      
target ccby  france attribution tatoebaorg  hybrid   happymidnight
predicted           
source 
target ccby  france attribution tatoebaorg  cm   yujihee
predicted           
source     
target ccby  france attribution tatoebaorg  ck   queserasera
predicted           
source    
target ccby  france attribution tatoebaorg  ck   rainbowus
predicted           
source      
target ccby  france attribution t